In [1]:
# Ultralytics 설치
!git clone https://github.com/ultralytics/yolov3
!cd yolov3; pip install -qr requirements.txt

Cloning into 'yolov3'...
remote: Enumerating objects: 9862, done.
remote: Total 9862 (delta 0), reused 0 (delta 0), pack-reused 9862
Receiving objects: 100% (9862/9862), 9.19 MiB | 24.39 MiB/s, done.
Resolving deltas: 100% (6667/6667), done.
     |████████████████████████████████| 636 kB 5.0 MB/s 


In [2]:
from IPython.display import Image, clear_output
import torch
clear_output()
print(f"Setup complete, Using torch {torch.__version__} \
  ({torch.cuda.get_device_properties(0).name \
    if torch.cuda.is_available() else 'CPU'})")

Setup complete, Using torch 1.9.0+cu102   (CPU)


# Oxford Pet Dataset 다운로드
- 다운로드 후 /content/data 디렉토리에 압축 해제

In [3]:
!wget https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
!wget https://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz

--2021-08-20 04:04:19--  https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 791918971 (755M) [application/x-gzip]
Saving to: ‘images.tar.gz’

images.tar.gz       100%[===================>] 755.23M  31.0MB/s    in 25s     

2021-08-20 04:04:45 (30.3 MB/s) - ‘images.tar.gz’ saved [791918971/791918971]

--2021-08-20 04:04:45--  https://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19173078 (18M) [application/x-gzip]
Saving to: ‘annotations.tar.gz’

annotations.tar.gz  100%[===================>]  18.28M  15.5MB/s    in 1.2s    

2021-08-20 04:04:46 (15.5 M

In [4]:
# /content/data 디렉토리 생성, 압축 해제
!mkdir /content/data
!tar -xvf /content/images.tar.gz -C /content/data
!tar -xvf /content/annotations.tar.gz -C /content/data

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
annotations/trimaps/._pomeranian_180.png
annotations/trimaps/pomeranian_180.png
annotations/trimaps/._pomeranian_181.png
annotations/trimaps/pomeranian_181.png
annotations/trimaps/._pomeranian_182.png
annotations/trimaps/pomeranian_182.png
annotations/trimaps/._pomeranian_183.png
annotations/trimaps/pomeranian_183.png
annotations/trimaps/._pomeranian_184.png
annotations/trimaps/pomeranian_184.png
annotations/trimaps/._pomeranian_185.png
annotations/trimaps/pomeranian_185.png
annotations/trimaps/._pomeranian_186.png
annotations/trimaps/pomeranian_186.png
annotations/trimaps/._pomeranian_187.png
annotations/trimaps/pomeranian_187.png
annotations/trimaps/._pomeranian_188.png
annotations/trimaps/pomeranian_188.png
annotations/trimaps/._pomeranian_189.png
annotations/trimaps/pomeranian_189.png
annotations/trimaps/._pomeranian_19.png
annotations/trimaps/pomeranian_19.png
annotations/trimaps/._pomeranian_190.png
annotations/trimaps/pomeranian_190.png
annota

# Dataset 리뷰 및  학습 / 검증 데이터 셋 분리
- Pandas DeataFrame으로 메타데이터 생성, 학습, 검증 데이터 분리

In [5]:
# Ultralytics Yolo images 와 LABELS 디렉토리를 train, val 용으로 생성
!mkdir /content/ox_pet;
!cd /content/ox_pet; mkdir images; mkdir labels;
!cd /content/ox_pet/images; mkdir train; mkdir val;
!cd /content/ox_pet/labels; mkdir train; mkdir val; 

In [7]:
import pandas as pd
a = pd.read_csv('/content/data/annotations/trainval.txt', sep=' ', 
            header=None, names=['img_name', 'class_id', 'etc1', 'etc2'])
#  a = a['img_name'].apply(lambda x: x[:x.rfind('_')])

In [8]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

# 전체 image/annotation 파일명을 가지고 리스트 파일명을 입력 받아 메타 파일용
# DataFrame 및 학습/검증 DataFrame 생성.
def make_train_valid_df(list_filepath, img_dir, anno_dir, test_size=0.1):
  pet_df = pd.read_csv(list_filepath, sep=' ', 
                       header=None, names=['img_name', 'class_id', 'etc1', 'etc2'])
  #class_name은 image 파일명에서 맨 마지막 '_' 문자 앞까지 해당
  pet_df['class_name'] = pet_df['img_name'].apply(lambda x: x[:x.rfind('_')])
  # image 파일명과 annotation 파일명의 절대경로 컬럼 추가
  pet_df['img_filepath'] = img_dir + pet_df['img_name'] + '.jpg'
  pet_df['anno_filepath'] = anno_dir + pet_df['img_name'] + '.xml'
  # annotation xml 파일이 없는데, trainval.txt에는 리스트가 있는 경우가 있음.
  # 해당 row를 삭제
  pet_df = remove_no_annos(pet_df)

  # 전체 데이터의 10%를 검증 데이터로 분리
  train_df, val_df = train_test_split(pet_df, test_size=test_size, 
                                      stratify=pet_df['class_id'],
                                      random_state=10)
  return pet_df, train_df, val_df

In [9]:
def remove_no_annos(df):
  remove_rows = list()
  for index, row in df.iterrows():
    anno_filepath = row['anno_filepath']
    if not os.path.exists(anno_filepath):
      print('##### index:', index, anno_filepath, '가 존재하지 않음. 삭제')
      #해당 DataFrame index를 remove_rows list에 담음
      remove_rows.append(index)
  df = df.drop(remove_rows, axis=0, inplace=False)
  return df

In [10]:
list_filepath = '/content/data/annotations/trainval.txt'
img_dir = '/content/data/images/'
anno_dir = '/content/data/annotations/xmls/'
pet_df, train_df, val_df = make_train_valid_df(list_filepath,
                                               img_dir, anno_dir,
                                               test_size=0.1)

##### index: 4 /content/data/annotations/xmls/Abyssinian_104.xml 가 존재하지 않음. 삭제
##### index: 262 /content/data/annotations/xmls/Bengal_111.xml 가 존재하지 않음. 삭제
##### index: 1456 /content/data/annotations/xmls/samoyed_10.xml 가 존재하지 않음. 삭제
##### index: 2128 /content/data/annotations/xmls/Bengal_175.xml 가 존재하지 않음. 삭제
##### index: 2395 /content/data/annotations/xmls/Egyptian_Mau_14.xml 가 존재하지 않음. 삭제
##### index: 2402 /content/data/annotations/xmls/Egyptian_Mau_156.xml 가 존재하지 않음. 삭제
##### index: 2427 /content/data/annotations/xmls/Egyptian_Mau_186.xml 가 존재하지 않음. 삭제
##### index: 3177 /content/data/annotations/xmls/Ragdoll_199.xml 가 존재하지 않음. 삭제
##### index: 3246 /content/data/annotations/xmls/saint_bernard_15.xml 가 존재하지 않음. 삭제


In [ ]:
pet_df.head()

In [ ]:
train_df.head()

In [ ]:
val_df.head()

# Oxford Pet Dataset의 annotation을 Ultralytics Yolo format으로 변환
- annotation용 xml파일을 txt파일로 변환
- 하나의 이미지는 하나의 txt파일로 변환
- 확장자를 제외한 이미지의 파일명과 annotation 파일명이 서로 동일 해야 함.
- voc annotation의 좌상단(top left : x1 y1), 우하단(bootom right: x2 y2)좌표를 Bounding box 중심 좌료(center_x, center_y)와 너비, 높이로 변경
- 중심좌표와 너비, 높이는 원본 이미지 레벨로 scale 되어야 함. 0~1사이 값으로 변환
- class_id는 여러개의 label들을 0부터 순차적으로 1씩 증가하여 id 부여

In [11]:
# Class 명을 부여, Class id는 자동적으로 
# CLASS_NAMES 개별 원소들을 순차적으로 0~ 36까지 부여
CLASS_NAMES = pet_df['class_name'].unique().tolist()
print(CLASS_NAMES)

['Abyssinian', 'american_bulldog', 'american_pit_bull_terrier', 'basset_hound', 'beagle', 'Bengal', 'Birman', 'Bombay', 'boxer', 'British_Shorthair', 'chihuahua', 'Egyptian_Mau', 'english_cocker_spaniel', 'english_setter', 'german_shorthaired', 'great_pyrenees', 'havanese', 'japanese_chin', 'keeshond', 'leonberger', 'Maine_Coon', 'miniature_pinscher', 'newfoundland', 'Persian', 'pomeranian', 'pug', 'Ragdoll', 'Russian_Blue', 'saint_bernard', 'samoyed', 'scottish_terrier', 'shiba_inu', 'Siamese', 'Sphynx', 'staffordshire_bull_terrier', 'wheaten_terrier', 'yorkshire_terrier']


In [30]:
import glob
import xml.etree.ElementTree as ET
# 1개의 voc xml 파일을 Yolo txt 포맷으로 변환
def xml_to_txt(input_xml_file, output_txt_file, object_name):
  #ElementTree로 입력xml파일 파싱
  tree = ET.parse(input_xml_file)
  root = tree.getroot()
  img_node = root.find('size')
  # img_node가 없다면 종료
  if img_node is None:
    return None
  # 원본 이미지의 너비와 높이 추출
  img_width = int(img_node.find('width').text)
  img_height = int(img_node.find('height').text)
  # xml 파일내의 있는 모든 object Element를 찾음
  value_str = None
  with open(output_txt_file, 'w') as output_fpointer:
    for obj in root.findall('object'):
      # bndbox를 찾아서 좌상단(xmin, ymin), 우하단(xmax, ymax) 추출
      xmlbox = obj.find('bndbox')
      x1 = int(xmlbox.find('xmin').text)
      y1 = int(xmlbox.find('ymin').text)
      x2 = int(xmlbox.find('xmax').text)
      y2 = int(xmlbox.find('ymax').text)
      # 좌표값이 0보다 작은 값이 있으면 종료
      if (x1<0) or (x2<0) or (y1<0) or (y2<0):
        break
      # ojbect_name과 원본 좌표를 입력,
      # Yolo포맷으로 변환하는 convert_yolo_coord() 호출
      class_id, cx_norm, cy_norm, w_norm, h_norm = \
        convert_yolo_coord(object_name, img_width, img_height, x1, y1, x2, y2)
      # 변환된 yolo 좌표를 object별로 출력 txt 파일에 write
      value_str = ('{0} {1} {2} {3} {4}') \
        .format(class_id, cx_norm, cy_norm, w_norm, h_norm)
      output_fpointer.write(value_str+'\n')
      # debugging용 출력
      # print(ojbect_name, value_str)

In [31]:
# object_name과 원본 좌표를 Yolo 포맷으로 변환 함수
def convert_yolo_coord(object_name, img_width, img_height, x1, y1, x2, y2):
  # class_id는 CLASS_NAMES 리스트에서 index 호출.
  class_id = CLASS_NAMES.index(object_name)
  # 중심좌표와 너비, 높이 계산
  center_x = (x1 + x2) / 2
  center_y = (y1 + y2) / 2
  width = x2 - x1
  height = y2 - y1
  # 원본 이미지 기준으로 중심 좌표와 너비, 높이 0~1 사이 값으로 scaling
  center_x_norm = center_x / img_width
  center_y_norm = center_y / img_height
  width_norm = width / img_width
  height_norm = height / img_height
  return class_id, round(center_x_norm, 7), round(center_y_norm, 7),\
    round(width_norm, 7), round(height_norm, 7)

In [32]:
class_id = CLASS_NAMES.index('yorkshire_terrier')
class_id

36

In [33]:
xml_to_txt('/content/data/annotations/xmls/Abyssinian_1.xml',
           '/content/ox_pet/labels/train/Abyssinian_1.txt', 'Abyssinian')

# VOC Format 의 여려개 xml 파일들을 Yolo format으로 변환 후 Ultralytics directory 구조로 입력
- VOC XML 파일들이 있는 디렉토리와 변환하여 출력될 Yolo format txt 파일들이 있는 디렉토리를 입력.


In [36]:
import shutil
def make_yolo_anno_file(df, tgt_images_dir, tgt_labels_dir):
  for index, row in df.iterrows():
    src_image_path = row['img_filepath']
    src_label_path = row['anno_filepath']
    # 이미지 1개당 1개의 오브젝트만 존재, class_name를 objec_name 설정
    object_name = row['class_name']
    # yolo format으로 annotation할 txt 파일의 절대 경로 지정.
    target_label_path = tgt_labels_dir + row['img_name'] + '.txt'
    # image의 경우 target images 디렉토리 단순 copy
    shutil.copy(src_image_path, tgt_images_dir)
    # annotation의 경우 xml 파일을 target labels 디렉토리에 
    # Ultralytics Yolo format으로 변환, 생성
    xml_to_txt(src_label_path, target_label_path, object_name)

In [37]:
make_yolo_anno_file(train_df, 
                    '/content/ox_pet/images/train/',
                    '/content/ox_pet/labels/train/')

In [39]:
make_yolo_anno_file(val_df,
                    '/content/ox_pet/images/val/',
                    '/content/ox_pet/labels/val/')

# Oxford Pet Dataset fit
- 생성된 디렉토리 구조에 맞춰서 dataset yaml 파일 생성

In [40]:
!wget -O /content/ox_pet/ox_pet.yaml http://raw.githubusercontent.com/chulminkw/DLCV/master/data/util/ox_pet.yaml

--2021-08-20 05:20:56--  http://raw.githubusercontent.com/chulminkw/DLCV/master/data/util/ox_pet.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://raw.githubusercontent.com/chulminkw/DLCV/master/data/util/ox_pet.yaml [following]
--2021-08-20 05:20:56--  https://raw.githubusercontent.com/chulminkw/DLCV/master/data/util/ox_pet.yaml
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 754 [text/plain]
Saving to: ‘/content/ox_pet/ox_pet.yaml’

/content/ox_pet/ox_ 100%[===================>]     754  --.-KB/s    in 0s      

2021-08-20 05:20:57 (35.5 MB/s) - ‘/content/ox_pet/ox_pet.yaml’ saved [754/754]



In [ ]:
CLASS_NAMES

### train시 1시간 이상 소모 됨.  colab환경에서 자동 클릭 처리를 해 줘서 세션을 유지 시켜 줘야 함.
### "shift + cntr + i" browser console open
### 꼭 학습 종료 후 노트죽 강제 종료 하기!!! 12시간 제한!!!
```
function ClickConnect() {
    console.log('Working');
    document.querySelector('colab-toolbar-button#connect').click();
}
setInterval(ClickConnect, 60000)
```

In [44]:
# 분석 결과 저장을 위해 내구글 드리이브 연결
import os, sys
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [45]:
# link 생성
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive

 품질재단세종_빅데이타_202104   ETC
 품질재단서울_빅데이터_202106   세종ICT_탠서플로우_주말_202107
'Colab Notebooks'	        MacBookPro15


In [46]:
# 내 구굴드라이브에 학습결과 저정 디렉토리 생성
!mkdir /mydrive/ultra_workdir

In [ ]:
# 학습
!cd /content/yolov3; python train.py --img 640 --batch 8 --epochs 20 \
  --data /content/ox_pet/ox_pet.yaml --weights yolov3.pt \
  --project=/mydrive/ultra_workdir --name pet \
  --exist-ok

github: up to date with https://github.com/ultralytics/yolov3 ✅
YOLOv3 🚀 v9.5.0-13-g1be3170 torch 1.9.0+cu102 CPU

Namespace(adam=False, artifact_alias='latest', batch_size=8, bbox_interval=-1, bucket='', cache_images=False, cfg='', data='/content/ox_pet/ox_pet.yaml', device='', entity=None, epochs=20, evolve=False, exist_ok=True, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[640, 640], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='pet', noautoanchor=False, nosave=False, notest=False, project='/mydrive/ultra_workdir', quad=False, rect=False, resume=False, save_dir='/mydrive/ultra_workdir/pet', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=8, upload_dataset=False, weights='yolov3.pt', workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir /mydrive/ultra_workdir', view at http://localhost:6006/
hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup

In [ ]:
# image 파일 inference
!cd yolov3; python detect.py --source /content/data/images/pug_100.jpg \
  --weights /mydrive/ultra_workdir/weights/best.pt \
  --conf 0.2 \
  --project=/content/data/output \
  --name=run_image \
  --exist-ok --line-thickness 2

In [ ]:
# 영상파일 inference
!wget /content/data/ox_dog.mp4 http://raw.githubusercontent.com/chulminkw/DLCV/master/data/video/ox_dog.mp4?raw=true
!cd yolov3; python detect.py --source /content/data/ox_dog.mp4 \
  --weights /mydrive/ultra_workdir/weights/best.pt \
  --conf 0.5 --iou 0.6 \
  --project=/content/data/output \
  --name=run_image \
  --exist-ok --line-thickness 2

In [ ]:
from IPython.display import Image
Image(filename='/content/data/images/boxer_100.jpg', width=400)

# test.py를 이용하여 Test 테이터를 Evalutation하기

In [ ]:
#Run YOLOv3 on COCO val2017
!cd yolov3; python test.py --weights /mydive/ultra_workdir/weights/best.pt \
  --data /content/ox_pet/ox_pet.yaml \
  --project /content/data/output --name test_result \
  --exist-ok --img 640 --iou 0.65

In [ ]:
Image(filename='/content/data/output/test_result/confusion_matrix.png', width=800)